<a href="https://colab.research.google.com/github/a-smaruj/Machine-learning-finance-classes/blob/use-case-1/MLF_class_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

### Load data

In [2]:
creditcard = pd.read_csv("creditcard.csv")
creditcard.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
len(creditcard)

284807

In [4]:
creditcard['Class'].unique()

array([0, 1])

In [5]:
creditcard = creditcard.dropna()

In [6]:
creditcard.groupby(['Class'])['Class'].count()

Class
0    284315
1       492
Name: Class, dtype: int64

In [7]:
creditcard.groupby(['Class'])['Class'].count() * 100 / len(creditcard)

Class
0    99.827251
1     0.172749
Name: Class, dtype: float64

### Train test split data

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    creditcard[creditcard.columns[~creditcard.columns.isin(['Class'])]],
    creditcard['Class'],
    test_size=0.33, random_state=42)

### K - value estimate

In [ ]:
k_values = [i for i in range (1,15)]
scores = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    score = cross_val_score(knn, X_train, y_train, cv=3)
    scores.append(np.mean(score))

In [ ]:
sns.lineplot(x = k_values, y = scores, marker = 'o')
plt.xlabel("K Values")
plt.ylabel("Accuracy Score")

### KNeighbors model

In [10]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [11]:
def score(y_test, y_pred):
  accuracy = accuracy_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  print("Accuracy:", accuracy, "\n"
        "Precision:", precision, "\n"
        "Recall:", recall)

  print("Confusion matrix: \n", confusion_matrix(y_test, y_pred))

In [12]:
score(y_test, y_pred)

Accuracy: 0.9985955504484663 
Precision: 1.0 
Recall: 0.11409395973154363
Confusion matrix: 
 [[93838     0]
 [  132    17]]


### Undersampling

In [13]:
rus = RandomUnderSampler()
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [14]:
knn.fit(X_train_rus, y_train_rus)
y_pred_rus = knn.predict(X_test)

In [15]:
score(y_test, y_pred_rus)

Accuracy: 0.6111164309957761 
Precision: 0.002732165787820005 
Recall: 0.6711409395973155
Confusion matrix: 
 [[57337 36501]
 [   49   100]]


In [16]:
undersample = NearMiss(version = 3, n_neighbors=3)
X_train_nm, y_train_nm = undersample.fit_resample(X_train, y_train)

In [17]:
knn.fit(X_train_nm, y_train_nm)
y_pred_nm = knn.predict(X_test)

In [18]:
score(y_test, y_pred_nm)

Accuracy: 0.6297147477842681 
Precision: 0.0026408703390073773 
Recall: 0.6174496644295302
Confusion matrix: 
 [[59093 34745]
 [   57    92]]


### Oversampling

In [19]:
oversample = SMOTE()
X_train_os, y_train_os = oversample.fit_resample(X_train, y_train)

In [20]:
knn.fit(X_train_os, y_train_os)
y_pred_os = knn.predict(X_test)

In [21]:
score(y_test, y_pred_os)

Accuracy: 0.9562918275931778 
Precision: 0.018955042527339004 
Recall: 0.5234899328859061
Confusion matrix: 
 [[89801  4037]
 [   71    78]]


### Pipeline

In [22]:
model = KNeighborsClassifier(n_neighbors=3)
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
#steps = [('over', over), ('under', under), ('model', model)]
pipeline = make_pipeline(over, under, model)

In [23]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96     93838
           1       0.01      0.49      0.02       149

    accuracy                           0.93     93987
   macro avg       0.50      0.71      0.49     93987
weighted avg       1.00      0.93      0.96     93987



### Weighted Logistic Regression

In [25]:
weights = {0:1.0, 1:1.0}
model = LogisticRegression(solver= 'lbfgs',
                           class_weight=weights)

In [26]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [27]:
score(y_test, y_pred)

Accuracy: 0.9989573026056795 
Precision: 0.6946564885496184 
Recall: 0.610738255033557
Confusion matrix: 
 [[93798    40]
 [   58    91]]


### Balanced Logistic Regression

In [28]:
model = LogisticRegression(solver= 'lbfgs',
                           class_weight='balanced')

In [29]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [30]:
score(y_test, y_pred)

Accuracy: 0.9666868822283933 
Precision: 0.04207616707616708 
Recall: 0.9194630872483222
Confusion matrix: 
 [[90719  3119]
 [   12   137]]
